# Radial Average
## Importing libraries

In [1]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from tqdm import tqdm
from scipy.special import erf
import pickle
import itertools

from SALib.sample import saltelli
from SALib.analyze import sobol

# Personal libraries
import sixtrackwrap as sx

from parameters import *

import time

## Computing

In [2]:
error_2 = {}
DA_2 = {}
raw_error_2 = {}
DA_5 = {}
error_5 = {}

alpha_preliminary_values = np.linspace(-1.0, 1.0, samples)
alpha_values = np.arccos(alpha_preliminary_values) / 2
d_preliminar_alpha = alpha_preliminary_values[1] - alpha_preliminary_values[0]

# Extracting the radiuses with theta1 = theta2 = 0.0

time_start = time.time()

engine = sx.radial_scanner(
    alpha_values, 
    np.zeros(alpha_values.shape),
    np.zeros(alpha_values.shape),
    d_r,
    starting_step=starting_step
)
engine.scan(turn_sampling[0], turn_sampling[-1], batch_size=batch_size)
all_radiuses = engine.extract_DA(turn_sampling)

elapsed_time_engine = time.time() - time_start


Active radiuses: 2049 / 2049
Sample size per active radius: 3
Expected execution time for step: nan
Elapsed time for whole iteration: 32.09
Time per single iteration: 5.221152084521866e-05
r: 12.0 . Turns to do: 100 . Min found: 100
Active radiuses: 2049 / 2049
Sample size per active radius: 3
Expected execution time for step: 32.09
Elapsed time for whole iteration: 32.08
Time per single iteration: 5.218078978802104e-05
r: 15.0 . Turns to do: 100 . Min found: 100
Active radiuses: 2049 / 2049
Sample size per active radius: 3
Expected execution time for step: 32.08
Elapsed time for whole iteration: 32.14
Time per single iteration: 5.228652708972976e-05
r: 18.0 . Turns to do: 100 . Min found: 100
Active radiuses: 2049 / 2049
Sample size per active radius: 3
Expected execution time for step: 32.14
Elapsed time for whole iteration: 31.99
Time per single iteration: 5.2034343687212326e-05
r: 21.0 . Turns to do: 100 . Min found: 50
Active radiuses: 2049 / 2049
Sample size per active radius: 3


### 2D simple integral

In [3]:
skips = [1]
while True:
    if (samples - 1) // skips[-1] > 4:
        skips.append(skips[-1] * 2)
    else:
        break

for i in skips:
    alpha = alpha_preliminary_values[::i]
    cutted_radiuses = all_radiuses[::i]
    value = integrate.simps(cutted_radiuses ** 2, alpha, axis=0)
    less_value = integrate.simps(cutted_radiuses[::2] ** 2, alpha[::2], axis=0)
    uncertainty = np.abs((value - less_value))

    DA = np.sqrt(value / 2)
    uncertainty = 0.5 * np.power(value / 2, -0.5) * uncertainty
    DA_5[cutted_radiuses.shape] = np.asarray(DA)
    error_5[cutted_radiuses.shape] = uncertainty 

### Actual angular averaging

In [6]:
time_start = time.time()

values = []
indexes = []
raw_values = []
refined_values = []
    
for i in tqdm(range(len(turn_sampling))):
    radiuses = all_radiuses[:, i]

    r, a, th1, th2 = sx.full_track_particles(
        radiuses,
        alpha_values,
        np.zeros(alpha_values.shape),
        np.zeros(alpha_values.shape),
        turn_sampling[i])

    count_matrix, avg_matrix, _ = sx.accumulate_and_return(r, a, th1, th2, n_subdivisions)
    
    if i == 0:
        with open(savepath + "data/matrices_2.pkl", 'wb') as f:
            pickle.dump((count_matrix, avg_matrix), f, protocol=4)
    
    _, _, result_total, validity_total = sx.recursive_accumulation(count_matrix, avg_matrix)
    
    values.append(result_total)
    indexes.append(np.argmax(validity_total, axis=0))
    refined_values.append([result_total[indexes[-1][i]][i] for i in range(len(indexes[-1]))])
    raw_values.append(np.average(np.power(r, 4), axis=1))
        
steps = [1]
while True:
    if (values[0][0].shape[0] - 1) / steps[-1] > 4:
        steps.append(steps[-1] * 2)
    else:
        break
    
for jump in steps:
    DA = []
    error = []
    DA_mc = []
    raw_error_mc = []
    error_mc = []
    for i in range(len(turn_sampling)):
        DA.append(np.power(integrate.simps(refined_values[i][::jump], alpha_preliminary_values[::jump]) * 0.5, 1/4))
        temp = np.power(integrate.simps(refined_values[i][::jump * 2], alpha_preliminary_values[::jump * 2]) * 0.5, 1/4)
        error.append(np.absolute(DA[-1] - temp))

        DA_mc.append(np.power(np.average(refined_values[i][::jump]), 1/4))
        raw_error_mc.append(np.std(refined_values[i][::jump]))
        error_mc.append(0.25 * np.power(DA_mc[-1], -3) * np.std(values[i][::jump]) / np.sqrt(np.size(values[i][::jump])))

    DA_2[( len(refined_values[i][::jump]), "refined", "int")] = DA
    error_2[( len(refined_values[i][::jump]), "refined", "int")] = error
    DA_2[( len(refined_values[i][::jump]), "refined", "mc")] = DA_mc
    error_2[( len(refined_values[i][::jump]), "refined", "mc")] = error_mc
    raw_error_2[( len(refined_values[i][::jump]), "refined", "mc")] = raw_error_mc

    DA = []
    error = []
    DA_mc = []
    raw_error_mc = []
    error_mc = []
    for i in range(len(turn_sampling)):
        DA.append(np.power(integrate.simps(raw_values[i][::jump], alpha_preliminary_values[::jump]) * 0.5, 1/4))
        temp = np.power(integrate.simps(raw_values[i][::jump * 2], alpha_preliminary_values[::jump * 2]) * 0.5, 1/4)
        error.append(np.absolute(DA[-1] - temp))

        DA_mc.append(np.power(np.average(raw_values[i][::jump]), 1/4))
        raw_error_mc.append(np.std(raw_values[i][::jump]))
        error_mc.append(0.25 * np.power(DA_mc[-1], -3) * np.std(values[i][::jump]) / np.sqrt(np.size(values[i][::jump])))

    DA_2[( len(raw_values[i][::jump]), "raw", "int")] = DA
    error_2[( len(raw_values[i][::jump]), "raw", "int")] = error
    DA_2[( len(raw_values[i][::jump]), "raw", "mc")] = DA_mc
    error_2[( len(raw_values[i][::jump]), "raw", "mc")] = error_mc
    raw_error_2[( len(raw_values[i][::jump]), "raw", "mc")] = raw_error_mc

    for j in range(len(values[0])):
        DA = []
        error = []
        DA_mc = []
        raw_error_mc = []
        error_mc = []
        for i in range(len(turn_sampling)):
            DA.append(np.power(integrate.simps(values[i][j][::jump], alpha_preliminary_values[::jump]) * 0.5, 1/4))
            temp = np.power(integrate.simps(values[i][j][::jump * 2], alpha_preliminary_values[::jump * 2]) * 0.5, 1/4)
            error.append(np.absolute(DA[-1] - temp))

            DA_mc.append(np.power(np.average(values[i][j][::jump]), 1/4))
            raw_error_mc.append(np.std(values[i][j][::jump]))
            error_mc.append(0.25 * np.power(DA_mc[-1], -3) * np.std(values[i][j][::jump]) / np.sqrt(np.size(values[i][j][::jump])))

        DA_2[( len(values[i][j][::jump]), 2 ** (j), "int")] = DA
        error_2[( len(values[i][j][::jump]), 2 ** (j), "int")] = error
        DA_2[( len(values[i][j][::jump]), 2 ** (j), "mc")] = DA_mc
        error_2[( len(values[i][j][::jump]), 2 ** (j), "mc")] = error_mc
        raw_error_2[( len(values[i][j][::jump]), 2 ** (j), "mc")] = raw_error_mc
        
elapsed_time_processing = time.time() - time_start


  0%|          | 0/5 [00:00<?, ?it/s]/home/carlidel/Insync/carlo.montanari3@studio.unibo.it/OneDrive Biz/optimized_code/sixtrackwrap/sixtrackwrap/__init__.py:68: RuntimeWarning: invalid value encountered in true_divide
  matrices = np.nansum(matrices.reshape(

100%|██████████| 5/5 [01:36<00:00, 19.25s/it]


## Saving data

In [17]:
with open(savepath + "data/DA_2.pkl", 'wb') as f:
    pickle.dump(DA_2, f, protocol=4)
    
with open(savepath + "data/error_2.pkl", 'wb') as f:
    pickle.dump(error_2, f, protocol=4)
    
with open(savepath + "data/raw_error_2.pkl", 'wb') as f:
    pickle.dump(raw_error_2, f, protocol=4)
    
with open(savepath + "data/DA_5.pkl", 'wb') as f:
    pickle.dump(DA_5, f, protocol=4)
    
with open(savepath + "data/error_5.pkl", 'wb') as f:
    pickle.dump(error_5, f, protocol=4)

with open(savepath + "data/time_2.pkl", "wb") as f:
    pickle.dump((elapsed_time_engine, elapsed_time_processing), f, protocol=4)